In [ ]:
# install gitPython
import os, sys, time
!pip install gitPython
# clone my repository
import git
!git clone https://github.com/sungbinlim/RLclass.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 7.0 MB/s eta 0:00:00
Cloning into 'RLclass'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 40 (delta 8), reused 32 (delta 0), pack-reused 0
Unpacking objects: 100% (40/40), 7.45 KiB | 693.00 KiB/s, done.


In [ ]:
%cd /content/RLclass/STAT436/grid_world/

/content/RLclass/STAT436/grid_world


In [ ]:
sys.path.append("/content/RLclass/STAT436/grid_world/")
from grid_world import *
from dynamics import *

## Hacking Dynamics

Bellman equation:

$$
\mathbf{v}=\mathbf{P}_{\text{reward}}\mathbf{r}+\gamma\mathbf{P}_{\text{value}}\mathbf{v}
$$

where $\mathbf{v}\in\mathbb{R}^{|\mathcal{S}|}$, $\mathbf{r}\in\mathbb{R}^{|\mathcal{R}|}$, $\mathbf{P}_{\text{reward}}\in\mathbb{R}^{|\mathcal{S}|\times|\mathcal{R}|}$, and $\mathbf{P}_{\text{value}}\in\mathbb{R}^{|\mathcal{S}|\times|\mathcal{S}|}$ such that 

\begin{aligned}
\left(\mathbf{P}_{\text{reward}}\right)_{jq}=\sum_{a_{p}\in\mathcal{A}}\pi(a_{p}|s_{j})\sum_{s_{i}\in\mathcal{S}}\mathcal{P}(s_{i},r_{q}|s_{j},a_{p})=\boldsymbol{\pi}_{p}\mathbf{P}_{i,j,p,q}\mathbf{1}_{i}\\
\left(\mathbf{P}_{\text{value}}\right)_{ji}=\sum_{a_{p}\in\mathcal{A}}\pi(a_{p}|s_{j})\sum_{r_{q}}\mathcal{P}(s_{i},r_{q}|s_{j},a_{p})=\boldsymbol{\pi}_{p}\mathbf{P}_{i,j,p,q}\mathbf{1}_{q}
\end{aligned}

If $\gamma \in (0, 1)$ then we can solve the equation as follows
$$
\mathbf{v}=(I-\gamma\mathbf{P}_{\text{value}})^{-1}\mathbf{P}_{\text{reward}}\mathbf{r}
$$

To compute action-value function $q_{\pi}(s,a)$, we use the following formula:
$$
q_{\pi}(s,a)=\sum_{s'\in\mathcal{S},r\in\mathcal{R}} [r +\gamma v_{\pi}(s')]\mathcal{P}(s',r|s,a)
$$

In [ ]:
import numpy as np
class pi_dynamics:
    def __init__(self, pi, gamma, reward, dynamics):
        self.pi = pi    
        self.gamma = gamma
        self.reward = reward
        grid_world_dynamics = dynamics()
        self.dynamics = grid_world_dynamics.dynamics
        self.pi_dynamics = np.zeros_like(self.dynamics) # [current_state, next_state, action, value] 
        self.P_reward = np.zeros((12, 3))
        self.P_value = np.zeros((12, 12))
        self.pi_dynamics, self.P_reward, self.P_value = self.update_all()

    def update_pi_dynamics(self, pi_dynamics):
        """
        compute pi * dynamics
        """
        for j in range(12):
            for p in range(4):
                # broadcasting
                pi_dynamics[j, :, p, :] = self.pi[j, p] * self.dynamics[j, :, p, :]
        return pi_dynamics

    def compute_P_reward(self, P_reward):
        """
        return P_reward[next_state, reward]: marginalize pi_dynamics in state
        """
        for j in range(12):
             for q in range(3):
                 # marginalization
                 P_reward[j, q] = np.sum(self.pi_dynamics[j, :, :, q])
        return P_reward

    def compute_P_value(self, P_value):
        """
        return P_value[next_state, state]: marginalize pi_dynamics in reward
        """
        # state -> state
        for j in range(12):
             for i in range(12):
                 # marginalization
                 P_value[j, i] = np.sum(self.pi_dynamics[j, i, :, :])
        return P_value

    def update_all(self):
        return self.update_pi_dynamics(self.pi_dynamics), self.compute_P_reward(self.P_reward), self.compute_P_value(self.P_value)

    def compute_state_value(self):
        """
        return state-value function via closed-form formula
        """
        coeff = np.eye(12) - self.gamma * self.P_value
        inv_coeff = np.linalg.inv(coeff)
        state_value = inv_coeff @ self.P_reward @ self.reward
        return state_value

    def compute_action_value(self):
        """
        return action-value function using state-value function
        """
        state_value = self.compute_state_value()
        expectation_reward = np.zeros((12, 4))
        expectation_value = np.zeros((12, 4))
        for i in range(12):
            for a in range(4):
                expectation_reward[i, a] = self.reward @ np.sum(self.dynamics, axis=1)[i, a, :]
                expectation_value[i, a] = self.gamma * state_value @ np.sum(self.dynamics, axis=3)[i, :, a]
        action_value = expectation_reward + expectation_value
        return action_value

## Let's run Grid World (using `randomAgent`)

In [ ]:
gamma = 0.99

# policy function
pi = np.array([0.25, 0.25, 0.25, 0.25]) #up, left, right, down
pi = np.reshape(np.tile(pi, 12), (12, 4))
# reward
reward = np.array([1, 0 ,-1])

# initialize dynamics with randomAgent
init_dynamics = dynamics
init_pi_dynamics = pi_dynamics(pi, gamma, reward, init_dynamics)
state_value = init_pi_dynamics.compute_state_value()
action_value = init_pi_dynamics.compute_action_value()

In [ ]:
# run random action
run_grid_world(pi, state_value, action_value)

Let's run grid world!
Success rate:28.000000000000004 %
-----------------
| ↑ | ↑ | ↑ | ↑ | 
-----------------
| ↑ | z | ↑ | ↑ | 
-----------------
| ↑ | ↑ | ↑ | ↑ | 
-----------------

state value:
 [-0.999 -0.931 -0.898 -0.238 -1.113  0.    -1.623 -2.238 -1.25  -1.396
 -1.592 -1.825]
action value:
 [[-0.982 -0.989 -0.922 -1.102]
 [-0.925 -0.989 -0.889 -0.922]
 [-0.827 -0.922 -0.235 -1.607]
 [-0.238 -0.238 -0.238 -0.238]
 [-1.011 -1.102 -1.102 -1.238]
 [ 0.     0.     0.     0.   ]
 [-1.093 -1.607 -2.215 -1.576]
 [-2.238 -2.238 -2.238 -2.238]
 [-1.144 -1.238 -1.382 -1.238]
 [-1.387 -1.238 -1.576 -1.382]
 [-1.604 -1.382 -1.807 -1.576]
 [-2.111 -1.576 -1.807 -1.807]]


## Policy Iteration with greedy policy

Let $\pi^{(0)}$ be an initial policy and set $q_{\pi^{(n)}}(s,a)$ be the action-value function of policy $\pi^{(n)}$ for $n=0,1,2,\ldots$ such that 

$$
\pi^{(n+1)}(a|s):=\mathbf{1}_{a=a_{n}(s)},\quad a_{n}(s):=\underset{a\in\mathcal{A}}{\text{argmax }}q_{\pi^{(n)}}(s,a)
$$

Then $q_{\pi^{(n)}} \to q_{\ast}$ as $n\to\infty$. 

In [ ]:
def one_hot(scalar, dim):
    """
    scalar -> one-hot vector 
    """
    vec = np.zeros(dim)
    vec[scalar] = 1
    return vec

# update policy w/ greedy policy
def update_policy(policy, action_value):
    """
    return greedy policy based on action-value function
    """
    greedy_policy = np.zeros_like(policy)

    for state in range(12):
        action = np.argmax(action_value[state, :])
        action = one_hot(action, 4)
        greedy_policy[state] = action
    
    return greedy_policy

# policy iteration
def policy_iteration(pi, gamma, reward, dynamics, eps=1e-8):
    """
    input: init_pi
    output: optimal_pi
    """

    # initial setting
    init_dynamics = dynamics
    dynamics_old = pi_dynamics(pi, gamma, reward, init_dynamics)
    state_value_old = dynamics_old.compute_state_value() 
    action_value_old = dynamics_old.compute_action_value()

    advances = np.inf
    n_it = 0

    while advances > eps:

        # policy improvement
        pi_new = update_policy(pi, action_value_old)
        dynamics_new = pi_dynamics(pi_new, gamma=gamma, reward=reward, dynamics=init_dynamics)
        
        # policy evaluation
        state_value_new = dynamics_new.compute_state_value()
        action_value_new = dynamics_new.compute_action_value()
        advances = np.sum(np.abs(state_value_new - state_value_old))
        n_it += 1

        # save policy and value functions
        pi = pi_new
        state_value_old = state_value_new
        action_value_old = action_value_new
                
    print("Policy iteration converged. (iteration={}, eps={})\n".format(n_it, np.sum(advances)))

    return pi_new, state_value_new, action_value_new

## Let's solve Grid World!

In [ ]:
# update policy via value function
start_time = time.time()
pi_new, state_value_new, action_value_new = policy_iteration(pi, 
                                                             gamma, 
                                                             reward, 
                                                             init_dynamics)
computation_time = time.time() - start_time
print("Wall-clock time for Policy Iteration: {} sec\n".format(np.round(computation_time, 4)))

Policy iteration converged. (iteration=4, eps=0.0)

Wall-clock time for Policy Iteration: 0.0326 sec



In [ ]:
# run updated policy
run_grid_world(pi_new, state_value_new, action_value_new)

Let's run grid world!
Success rate:99.0 %
-----------------
| → | → | → | ↑ | 
-----------------
| ↑ | z | ↑ | ↑ | 
-----------------
| ↑ | → | ↑ | ← | 
-----------------

state value:
 [15.244 15.398 15.554 15.711 15.054  0.    15.179 13.711 14.86  14.803
 14.953 14.803]
action value:
 [[15.107 15.092 15.244 14.904]
 [15.245 15.092 15.398 15.244]
 [15.399 15.244 15.554 15.027]
 [15.711 15.711 15.711 15.711]
 [15.054 14.904 14.904 14.711]
 [ 0.     0.     0.     0.   ]
 [15.179 15.027 13.574 14.803]
 [13.711 13.711 13.711 13.711]
 [14.86  14.711 14.655 14.711]
 [14.675 14.711 14.803 14.655]
 [14.953 14.655 14.655 14.803]
 [13.805 14.803 14.655 14.655]]


## Value Iteration
For each $n=0, 1, 2,...$, we set

$$
q^{(n+1)}(s,a) =\mathcal{M}(q^{(n)})(s,a):= \sum_{s',r}[r+\gamma \max_{a'\in\mathcal{A}}q^{(n)}(s',a')]\mathcal{P}(s',r|s,a)
$$
Then by the Banach fixed point theorem, we get
$$
q^{\star}(s,a)=\lim_{n\to\infty}q^{(n)}(s,a),\quad q^{\star}(s,a)=\mathcal{M}(q^{\star})(s,a)
$$
We will implement `Value Iteration` with the following tensors
$$
\mathbf{Q}_{\text{reward}}(s,a) = \sum_{s',r}r\cdot\mathcal{P}(s',r|s,a),\quad \mathbf{Q}_{\text{value}}(s,a)=\sum_{s',r}\gamma\max_{a'\in\mathcal{A}}q(s',a')\cdot\mathcal{P}(s',r|s,a)
$$

In [ ]:
def compute_Q_reward(reward, dynamics, broadcast=False):
    """
    return Q_reward[state, action]
    """
    Q_reward = np.zeros((12, 4))
    
    if not broadcast:
        for a in range(4):
            for i in range(12):
                Q_reward[i, a] = np.sum(dynamics[i, :, a, :] @ reward) # state, next_state, action, reward
    else:
        dynamics = dynamics.transpose((0, 2, 1, 3)) # state, action, next_state, reward
        dynamics = dynamics.reshape(-1, 12, 3)
        Q_reward = Q_reward.squeeze()
        Q_reward = np.sum(dynamics @ reward, axis=1)
        Q_reward = Q_reward.reshape((12, 4))

    return Q_reward

def compute_Q_value(action_value, dynamics, gamma, broadcast=False):
    """
    return Q_value[state, action]
    WARNING: very slow if you don't use broadcasting!
    """
    Q_value = np.zeros((12, 4))

    if not broadcast:
        for i in range(12):
                for a in range(4):
                    Q_value[i, a] = gamma * np.max(action_value, 1) @ np.sum(dynamics, axis=3)[i, :, a]
    else:
        dynamics = np.sum(dynamics, axis=3)
        dynamics = np.transpose(dynamics, (0, 2, 1)) # state, action, next_state)
        dynamics = dynamics.reshape(-1, 12)
        Q_value = Q_value.squeeze()
        Q_value = gamma * dynamics @ np.max(action_value, 1)
        Q_value = Q_value.reshape((12, 4))    
        
    return Q_value

# value iteration
def value_iteration(init_action_value, gamma, reward, dynamics, eps=1e-8, broadcast=False):
    """
    input: init action-value
    output: optimal action-value
    """

    # initial setting
    init_dynamics = dynamics() 
    init_dynamics = init_dynamics.dynamics
    action_value = init_action_value # state, action
    
    Q_reward = compute_Q_reward(reward, init_dynamics, broadcast)

    advances = np.inf
    n_it = 0

    while advances > eps or n_it <= 3:

        Q_value = compute_Q_value(action_value, init_dynamics, gamma, broadcast)
        new_action_value = Q_value + Q_reward
        advances = np.sum(np.abs(new_action_value - action_value))
        action_value = new_action_value
         
        n_it += 1

    print("Value iteration converged. (iteration={}, eps={})".format(n_it, np.sum(advances)))

    return new_action_value

In [ ]:
start_time = time.time()
init_action_value = action_value
optimal_action_value = value_iteration(init_action_value, gamma, reward, init_dynamics, eps=1e-5)
computation_time = time.time() - start_time
print("Wall-clock time for Value Iteration without broadcasting: {} sec\n".format(np.round(computation_time, 4)))

Value iteration converged. (iteration=1341, eps=9.995866980361257e-06)
Wall-clock time for Value Iteration without broadcasting: 1.7266 sec



In [ ]:
start_time = time.time()
init_action_value = action_value
optimal_action_value = value_iteration(init_action_value, gamma, reward, init_dynamics, eps=1e-5, broadcast=True)
computation_time = time.time() - start_time
print("Wall-clock time for Value Iteration with broadcasting: {} sec\n".format(np.round(computation_time, 4)))

Value iteration converged. (iteration=1341, eps=9.995866999901182e-06)
Wall-clock time for Value Iteration with broadcasting: 0.056 sec



In [ ]:
pi = np.array([0.25, 0.25, 0.25, 0.25]) #up, left, right, down
pi = np.reshape(np.tile(pi, 12), (12, 4))
pi_optimal = update_policy(pi, optimal_action_value) # update policy 
optimal_state_value = np.max(optimal_action_value, axis=1) 

In [ ]:
run_grid_world(pi_optimal, optimal_state_value, optimal_action_value)

Let's run grid world!
Success rate:99.0 %
-----------------
| → | → | → | ↑ | 
-----------------
| ↑ | z | ↑ | ↑ | 
-----------------
| ↑ | → | ↑ | ← | 
-----------------

state value:
 [15.244 15.398 15.554 15.711 15.054  0.    15.179 13.711 14.86  14.803
 14.953 14.803]
action value:
 [[15.107 15.092 15.244 14.904]
 [15.245 15.092 15.398 15.244]
 [15.399 15.244 15.554 15.027]
 [15.711 15.711 15.711 15.711]
 [15.054 14.904 14.904 14.711]
 [ 0.     0.     0.     0.   ]
 [15.179 15.027 13.574 14.803]
 [13.711 13.711 13.711 13.711]
 [14.86  14.711 14.655 14.711]
 [14.675 14.711 14.803 14.655]
 [14.953 14.655 14.655 14.803]
 [13.805 14.803 14.655 14.655]]
